In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")

In [8]:
# --------------------
# 📌 1. Connexion à PostgreSQL
# --------------------
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Configuration de connexion
USER = "postgres"
PASSWORD = "159753"
HOST = "localhost"
PORT = "5432"
DB = "SupplyChain_DW"

# Création de l'engine SQLAlchemy
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}')

In [9]:
# === Requête SQL ===
query = """
SELECT *
  FROM public."Dim Produit";
"""

In [10]:
# === Chargement dans un DataFrame pandas ===
df_prod = pd.read_sql(query, engine)

# === Affichage rapide ===
print(df_prod.head())

   id_produit_PK  Product_ID  \
0              1  PR-2846178   
1              2  PR-2760346   
2              3  PR-2761724   
3              4  PR-2760353   
4              5  PR-2760361   

                                        Product_Name    Brand  Price  \
0  Hair Density Boosting Scalp Serum for Thinning...  AAVRANI   30.0   
1  Hair Density Boosting Scalp Serum for Thinning...  AAVRANI   48.0   
2  Deep Conditioning Hair Mask for Dry, Damaged Hair  AAVRANI   39.0   
3  Mini Strengthening Hair Oil & Growth Serum Gif...  AAVRANI   35.0   
4  Hair and Scalp Recovery Pre-Wash Oil for Stren...  AAVRANI   35.0   

                             Size_Volume  Average_Rating  \
0  Size: .88 oz / 26 ml - 2 month supply          4.5242   
1    Size: 2 oz / 60 ml - 4 Month Supply          4.5242   
2                           Size: 8.4 oz          4.5659   
3                                                 4.6772   
4                           Size: 1.7 oz          4.7087   

             

In [11]:
df_prod

,id_produit_PK,Product_ID,Product_Name,Brand,Price,Size_Volume,Average_Rating,Temperature,Stocker_sans_lumiere,Categorie
0,1,PR-2846178,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,30.0,Size: .88 oz / 26 ml - 2 month supply,4.5242,4-8°C (Réfrigéré),Oui,Cheveux
1,2,PR-2760346,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,48.0,Size: 2 oz / 60 ml - 4 Month Supply,4.5242,4-8°C (Réfrigéré),Oui,Cheveux
2,3,PR-2761724,"Deep Conditioning Hair Mask for Dry, Damaged Hair",AAVRANI,39.0,Size: 8.4 oz,4.5659,15-25°C (Température ambiante),Non,Cheveux
3,4,PR-2760353,Mini Strengthening Hair Oil & Growth Serum Gif...,AAVRANI,35.0,,4.6772,4-8°C (Réfrigéré),Oui,Cheveux
4,5,PR-2760361,Hair and Scalp Recovery Pre-Wash Oil for Stren...,AAVRANI,35.0,Size: 1.7 oz,4.7087,15-25°C (Température ambiante),Oui,Cheveux
...,...,...,...,...,...,...,...,...,...,...
19281,19282,PR-2591253,Lotus Pear Eau de Parfum,The 7 Virtues,32.0,Size: 0.33 oz / 11 mL,4.3926,15-25°C (Température ambiante),Non,Parfums
19282,19283,PR-2483253,Lotus Pear Eau de Parfum,The 7 Virtues,94.0,Size: 1.7 oz/ 50 mL eau de parfum spray,4.3926,15-25°C (Température ambiante),Non,Parfums
19283,19284,PR-2409241,Santal Vanille Gemstone Perfume Oil,The 7 Virtues,44.0,Size: 0.67 oz/ 20 mL,4.6491,15-25°C (Température ambiante),Oui,Cheveux
19284,19285,PR-2483279,Lotus Pear Perfume Oil,The 7 Virtues,44.0,Size: .67 oz / 20 mL Perfume Oil,4.4375,15-25°C (Température ambiante),Oui,Cheveux


In [12]:
# === Requête SQL ===
query = """
SELECT "id_produit_FK", "id_Date_FK", "id_Date_FK1", "id_mp_FK1", "id_mp_FK2", "id_mp_FK3"
	FROM public."Fact_Production";
"""
# === Chargement dans un DataFrame pandas ===
df_FactProd = pd.read_sql(query, engine)
df_FactProd.head(5)

,id_produit_FK,id_Date_FK,id_Date_FK1,id_mp_FK1,id_mp_FK2,id_mp_FK3
0,1,8185,8186,1017,581,25
1,2,4993,4998,916,662,799
2,3,2475,2478,963,1061,57
3,4,2581,2582,470,181,78
4,5,6513,6518,260,246,1102


In [13]:
# === Requête SQL ===
query = """
SELECT "id_Date_pk", fulldate
	FROM public."Dim_Date";
"""

In [14]:
# === Chargement dans un DataFrame pandas ===
df_date = pd.read_sql(query, engine)

# === Affichage rapide ===
print(df_date.head())

   id_Date_pk            fulldate
0           1 2022-01-01 00:00:00
1           2 2022-01-01 01:00:00
2           3 2022-01-01 02:00:00
3           4 2022-01-01 03:00:00
4           5 2022-01-01 04:00:00


In [15]:
# Étape 1 : Jointure entre produits et faits de production
Production = pd.merge(
   df_prod,
    df_FactProd,
    left_on='id_produit_PK',
    right_on='id_produit_FK',
    how='inner'
)

# Étape 2 : Jointure pour ajouter la date de production (id_Date_FK)
Production = Production.merge(
    df_date.rename(columns={"id_Date_pk": "id_Date_FK", "fulldate": "production_date"}),
    on="id_Date_FK",
    how="left"
)

# Étape 3 : Jointure pour ajouter la date de fin de production (id_Date_FK1)
Production = Production.merge(
   df_date.rename(columns={"id_Date_pk": "id_Date_FK1", "fulldate": "end_production_date"}),
    on="id_Date_FK1",
    how="left"
)

# Étape 4 : Suppression des colonnes FK de dates (devenues inutiles)
Production.drop(columns=["id_Date_FK", "id_Date_FK1", "id_produit_FK" , "Stocker_sans_lumiere", "Temperature" ], inplace=True)


In [16]:
Production

,id_produit_PK,Product_ID,Product_Name,Brand,Price,Size_Volume,Average_Rating,Categorie,id_mp_FK1,id_mp_FK2,id_mp_FK3,production_date,end_production_date
0,1,PR-2846178,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,30.0,Size: .88 oz / 26 ml - 2 month supply,4.5242,Cheveux,1017,581,25,2022-12-08 00:00:00,2022-12-08 01:00:00
1,2,PR-2760346,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,48.0,Size: 2 oz / 60 ml - 4 Month Supply,4.5242,Cheveux,916,662,799,2022-07-28 00:00:00,2022-07-28 05:00:00
2,3,PR-2761724,"Deep Conditioning Hair Mask for Dry, Damaged Hair",AAVRANI,39.0,Size: 8.4 oz,4.5659,Cheveux,963,1061,57,2022-04-14 02:00:00,2022-04-14 05:00:00
3,4,PR-2760353,Mini Strengthening Hair Oil & Growth Serum Gif...,AAVRANI,35.0,,4.6772,Cheveux,470,181,78,2022-04-18 12:00:00,2022-04-18 13:00:00
4,5,PR-2760361,Hair and Scalp Recovery Pre-Wash Oil for Stren...,AAVRANI,35.0,Size: 1.7 oz,4.7087,Cheveux,260,246,1102,2022-09-29 08:00:00,2022-09-29 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19087,19282,PR-2591253,Lotus Pear Eau de Parfum,The 7 Virtues,32.0,Size: 0.33 oz / 11 mL,4.3926,Parfums,136,953,775,2022-10-11 17:00:00,2022-10-11 22:00:00
19088,19283,PR-2483253,Lotus Pear Eau de Parfum,The 7 Virtues,94.0,Size: 1.7 oz/ 50 mL eau de parfum spray,4.3926,Parfums,253,188,463,2022-11-16 13:00:00,2022-11-16 18:00:00
19089,19284,PR-2409241,Santal Vanille Gemstone Perfume Oil,The 7 Virtues,44.0,Size: 0.67 oz/ 20 mL,4.6491,Cheveux,392,72,2,2023-02-19 20:00:00,2023-02-19 22:00:00
19090,19285,PR-2483279,Lotus Pear Perfume Oil,The 7 Virtues,44.0,Size: .67 oz / 20 mL Perfume Oil,4.4375,Cheveux,274,180,1088,2022-09-28 02:00:00,2022-09-28 04:00:00


In [17]:
# Assurer que les colonnes de date sont bien en format datetime
Production['production_date'] = pd.to_datetime(Production['production_date'])
Production['end_production_date'] = pd.to_datetime(Production['end_production_date'])

# Calcul de la durée de production
Production['production_duration'] = Production['end_production_date'] - Production['production_date']

# Optionnel : convertir la durée en heures (ou minutes, selon le besoin)
Production['production_duration_hours'] = Production['production_duration'].dt.total_seconds() / 3600

In [18]:
Production

,id_produit_PK,Product_ID,Product_Name,Brand,Price,Size_Volume,Average_Rating,Categorie,id_mp_FK1,id_mp_FK2,id_mp_FK3,production_date,end_production_date,production_duration,production_duration_hours
0,1,PR-2846178,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,30.0,Size: .88 oz / 26 ml - 2 month supply,4.5242,Cheveux,1017,581,25,2022-12-08 00:00:00,2022-12-08 01:00:00,0 days 01:00:00,1.0
1,2,PR-2760346,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,48.0,Size: 2 oz / 60 ml - 4 Month Supply,4.5242,Cheveux,916,662,799,2022-07-28 00:00:00,2022-07-28 05:00:00,0 days 05:00:00,5.0
2,3,PR-2761724,"Deep Conditioning Hair Mask for Dry, Damaged Hair",AAVRANI,39.0,Size: 8.4 oz,4.5659,Cheveux,963,1061,57,2022-04-14 02:00:00,2022-04-14 05:00:00,0 days 03:00:00,3.0
3,4,PR-2760353,Mini Strengthening Hair Oil & Growth Serum Gif...,AAVRANI,35.0,,4.6772,Cheveux,470,181,78,2022-04-18 12:00:00,2022-04-18 13:00:00,0 days 01:00:00,1.0
4,5,PR-2760361,Hair and Scalp Recovery Pre-Wash Oil for Stren...,AAVRANI,35.0,Size: 1.7 oz,4.7087,Cheveux,260,246,1102,2022-09-29 08:00:00,2022-09-29 13:00:00,0 days 05:00:00,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19087,19282,PR-2591253,Lotus Pear Eau de Parfum,The 7 Virtues,32.0,Size: 0.33 oz / 11 mL,4.3926,Parfums,136,953,775,2022-10-11 17:00:00,2022-10-11 22:00:00,0 days 05:00:00,5.0
19088,19283,PR-2483253,Lotus Pear Eau de Parfum,The 7 Virtues,94.0,Size: 1.7 oz/ 50 mL eau de parfum spray,4.3926,Parfums,253,188,463,2022-11-16 13:00:00,2022-11-16 18:00:00,0 days 05:00:00,5.0
19089,19284,PR-2409241,Santal Vanille Gemstone Perfume Oil,The 7 Virtues,44.0,Size: 0.67 oz/ 20 mL,4.6491,Cheveux,392,72,2,2023-02-19 20:00:00,2023-02-19 22:00:00,0 days 02:00:00,2.0
19090,19285,PR-2483279,Lotus Pear Perfume Oil,The 7 Virtues,44.0,Size: .67 oz / 20 mL Perfume Oil,4.4375,Cheveux,274,180,1088,2022-09-28 02:00:00,2022-09-28 04:00:00,0 days 02:00:00,2.0


# Prédire la durée de production (en heures)

In [19]:
Production['production_date'] = pd.to_datetime(Production['production_date'])
Production['production_hour'] = Production['production_date'].dt.hour
Production['production_weekday'] = Production['production_date'].dt.weekday

In [20]:
import re
import numpy as np

def extract_volume(val):
    if pd.isnull(val):
        return np.nan
    match = re.search(r"(\d+(?:\.\d+)?)\s*(mL|ml|oz)", val)
    if match:
        try:
            value = float(match.group(1))
            unit = match.group(2).lower()
            if 'oz' in unit:
                return round(value * 29.5735, 2)  # Convert oz to mL
            else:
                return value
        except ValueError:
            return np.nan
    return np.nan

Production['volume_ml'] = Production['Size_Volume'].apply(extract_volume)


In [21]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Étape 1 : Préparation des données
features = ['production_hour', 'production_weekday', 'volume_ml']
Production = Production.dropna(subset=features + ['production_duration_hours'])

X = Production[features]
y = Production['production_duration_hours']

# Étape 2 : Préprocessing + modèle
# Ici, on applique un standard scaler simple aux colonnes numériques (optionnel mais utile pour les gradients)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        random_state=42
    ))
])

# Étape 3 : Split + entraînement
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Étape 4 : Prédictions + évaluation
y_pred = pipeline.predict(X_test)

print("MAE :", mean_absolute_error(y_test, y_pred))
print("R² score :", r2_score(y_test, y_pred))


MAE : 1.2218650942553546
R² score : -0.02978529041523159


# Prédire si un produit risque un retard de production (problème de classification binaire)

In [22]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- 1. Création de la variable cible ---
Production['is_delayed'] = Production['production_duration_hours'] > 2

# --- 2. Définition des features et de la cible ---
features = ['Categorie', 'Brand', 'Price']
target = 'is_delayed'

# --- 3. Nettoyage : suppression des valeurs manquantes ---
Production = Production.dropna(subset=features + [target])

X = Production[features]
y = Production[target].astype(int)

# --- 4. Colonnes catégorielles ---
categorical = ['Categorie', 'Brand']

# --- 5. Préprocesseur ---
encoder = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[('cat', encoder, categorical)],
    remainder='passthrough'
)

# --- 6. Pipeline de modélisation ---
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# --- 7. Division train/test ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 8. Entraînement ---
pipeline.fit(X_train, y_train)

# --- 9. Évaluation ---
from sklearn.metrics import classification_report, confusion_matrix
y_pred = pipeline.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# --- 10. Sauvegarde du modèle ET de l'encodeur ---
try:
    joblib.dump(pipeline, "produitdelayed_model.pkl")
    joblib.dump(encoder, "categorie_brand_encoder.pkl")  # Sauvegarde séparée
    print("\n✅ Modèle et encodeur sauvegardés avec succès.")
except Exception as e:
    print(f"\n❌ Erreur lors de la sauvegarde : {e}")


Confusion Matrix:
 [[ 420  920]
 [ 614 1340]]
Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.31      0.35      1340
           1       0.59      0.69      0.64      1954

    accuracy                           0.53      3294
   macro avg       0.50      0.50      0.49      3294
weighted avg       0.52      0.53      0.52      3294


✅ Modèle et encodeur sauvegardés avec succès.


In [85]:
Production

,id_produit_PK,Product_ID,Product_Name,Brand,Price,Size_Volume,Average_Rating,Categorie,production_date,end_production_date,production_duration,production_duration_hours,production_hour,production_weekday,volume_ml,is_delayed
0,1,PR-2846178,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,30.0,Size: .88 oz / 26 ml - 2 month supply,4.5242,Cheveux,2022-12-08 00:00:00,2022-12-08 01:00:00,0 days 01:00:00,1.0,0,3,2602.47,False
1,2,PR-2760346,Hair Density Boosting Scalp Serum for Thinning...,AAVRANI,48.0,Size: 2 oz / 60 ml - 4 Month Supply,4.5242,Cheveux,2022-07-28 00:00:00,2022-07-28 05:00:00,0 days 05:00:00,5.0,0,3,59.15,True
2,3,PR-2761724,"Deep Conditioning Hair Mask for Dry, Damaged Hair",AAVRANI,39.0,Size: 8.4 oz,4.5659,Cheveux,2022-04-14 02:00:00,2022-04-14 05:00:00,0 days 03:00:00,3.0,2,3,248.42,True
4,5,PR-2760361,Hair and Scalp Recovery Pre-Wash Oil for Stren...,AAVRANI,35.0,Size: 1.7 oz,4.7087,Cheveux,2022-09-29 08:00:00,2022-09-29 13:00:00,0 days 05:00:00,5.0,8,3,50.27,True
5,6,PR-2797074,Jelly Clarifying Detox Shampoo for Scalp & Hair,AAVRANI,34.0,Size: 8.4 oz / 250 ml,4.7414,Cheveux,2023-01-03 04:00:00,2023-01-03 06:00:00,0 days 02:00:00,2.0,4,1,248.42,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19087,19282,PR-2591253,Lotus Pear Eau de Parfum,The 7 Virtues,32.0,Size: 0.33 oz / 11 mL,4.3926,Parfums,2022-10-11 17:00:00,2022-10-11 22:00:00,0 days 05:00:00,5.0,17,1,9.76,True
19088,19283,PR-2483253,Lotus Pear Eau de Parfum,The 7 Virtues,94.0,Size: 1.7 oz/ 50 mL eau de parfum spray,4.3926,Parfums,2022-11-16 13:00:00,2022-11-16 18:00:00,0 days 05:00:00,5.0,13,2,50.27,True
19089,19284,PR-2409241,Santal Vanille Gemstone Perfume Oil,The 7 Virtues,44.0,Size: 0.67 oz/ 20 mL,4.6491,Cheveux,2023-02-19 20:00:00,2023-02-19 22:00:00,0 days 02:00:00,2.0,20,6,19.81,False
19090,19285,PR-2483279,Lotus Pear Perfume Oil,The 7 Virtues,44.0,Size: .67 oz / 20 mL Perfume Oil,4.4375,Cheveux,2022-09-28 02:00:00,2022-09-28 04:00:00,0 days 02:00:00,2.0,2,2,1981.42,False


In [89]:
import sklearn
print(sklearn.__version__)

1.3.0
